# Data Simulation (Bayesian)

The purpose of this notebook is to write general functions which can then be invoked for conducting the bayesian data simulation. It therefore is some kind of test field where each function is built on it's one, start from kernel construction, simulation, and so on until a whole data set has been simulated.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import gp_utilities as gp_utils
from gp_utilities import *
import os
import logging
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

logging.disable(logging.WARNING)
# NEW
import tensorflow as tf
#tf.random.set_seed(42)

# COMPAT
#import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
tfk = tfp.math.psd_kernels

In [3]:
# CHECK :)
def get_kernel_bayesian(smooth_amplitude, smooth_length_scale,
                    periodic_amplitude, periodic_length_scale, periodic_period, periodic_local_amplitude, periodic_local_length_scale,
                    global_periodic_amplitude, global_periodic_length_scale, global_periodic_period,
                    irregular_amplitude, irregular_length_scale, irregular_scale_mixture,
                    matern_onehalf_amplitude, matern_onehalf_length_scale,
                    matern_threehalves_amplitude, matern_threehalves_length_scale,
                    matern_fivehalves_amplitude, matern_fivehalves_length_scale):
    
    # Smooth kernel
    smooth_kernel = tfk.ExponentiatedQuadratic(
        amplitude=smooth_amplitude,
        length_scale=smooth_length_scale)
    
    # Local periodic kernel
    local_periodic_kernel = (
        tfk.ExpSinSquared(
            amplitude=periodic_amplitude, 
            length_scale=periodic_length_scale,
            period=periodic_period) * 
        tfk.ExponentiatedQuadratic(
            amplitude=periodic_local_amplitude,
            length_scale=periodic_local_length_scale))
    
    # Periodic
    global_periodic_kernel = tfk.ExpSinSquared(
            amplitude=global_periodic_amplitude, 
            length_scale=global_periodic_length_scale,
            period=global_periodic_period)
    
    # Irregular kernel
    irregular_kernel = tfk.RationalQuadratic(
        amplitude=irregular_amplitude,
        length_scale=irregular_length_scale,
        scale_mixture_rate=irregular_scale_mixture)
    
    # Matern 1/2
    matern_onehalf_kernel = tfk.MaternOneHalf(
        amplitude = matern_onehalf_amplitude,
        length_scale = matern_onehalf_length_scale
    )
    
    # Matern 3/2
    matern_fivehalf_kernel = tfk.MaternThreeHalves(
        amplitude = matern_threehalves_amplitude,
        length_scale = matern_threehalves_length_scale
    )
    
    # Matern 5/2
    matern_fivehalf_kernel = tfk.MaternFiveHalves(
        amplitude = matern_fivehalves_amplitude,
        length_scale = matern_fivehalves_length_scale
    )
    
    return smooth_kernel + local_periodic_kernel + irregular_kernel + matern_onehalf_kernel + matern_fivehalf_kernel + matern_fivehalf_kernel

In [4]:
# CHECK :)
def build_gp(X_train, Y_train):

    #X_train = np.array(X_train).astype(float).reshape(-1, 1)
    #Y_train = np.array(Y_train)
    
    def build_gp_internal(smooth_amplitude, smooth_length_scale,
              periodic_amplitude, periodic_length_scale, periodic_period, periodic_local_amplitude, periodic_local_length_scale,
              global_periodic_amplitude, global_periodic_length_scale, global_periodic_period,
              irregular_amplitude, irregular_length_scale, irregular_scale_mixture,
              matern_onehalf_amplitude, matern_onehalf_length_scale,
              matern_threehalves_amplitude, matern_threehalves_length_scale,
              matern_fivehalves_amplitude, matern_fivehalves_length_scale,
              observation_noise_variance):
        """Defines the conditional dist. of GP outputs, given kernel parameters."""
        
        # Create the covariance kernel, which will be shared between the prior (which we
        # use for maximum likelihood training) and the posterior (which we use for
        # posterior predictive sampling)
        kernel = get_kernel_bayesian(smooth_amplitude, smooth_length_scale,
                  periodic_amplitude, periodic_length_scale, periodic_period, periodic_local_amplitude, periodic_local_length_scale,
                  global_periodic_amplitude, global_periodic_length_scale, global_periodic_period,
                  irregular_amplitude, irregular_length_scale, irregular_scale_mixture,
                  matern_onehalf_amplitude, matern_onehalf_length_scale,
                  matern_threehalves_amplitude, matern_threehalves_length_scale,
                  matern_fivehalves_amplitude, matern_fivehalves_length_scale)
        
        # Create the GP prior distribution, which we will use to train the model
        # parameters.
        return tfd.GaussianProcess(
          mean_fn=lambda x: np.mean(Y_train),
          kernel=kernel,
          index_points=X_train,
          observation_noise_variance=observation_noise_variance)
    
    return build_gp_internal

In [5]:
# CHECK :)
def prior_joint_model(X_train, Y_train, prior_dict=None):
    
    if prior_dict==None:
        prior_dict = {
            'smooth_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'smooth_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'periodic_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'periodic_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'periodic_period': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'periodic_local_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'periodic_local_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'global_periodic_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'global_periodic_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'global_periodic_period': tfd.LogNormal(loc=2., scale=np.float64(1)),
            'irregular_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'irregular_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'irregular_scale_mixture': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_onehalf_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_onehalf_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_threehalves_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_threehalves_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_fivehalves_amplitude': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'matern_fivehalves_length_scale': tfd.LogNormal(loc=0., scale=np.float64(1)),
            'observation_noise_variance': tfd.LogNormal(loc=0., scale=np.float64(1)),
        }
    
    prior_gp = build_gp(X_train, Y_train)
    prior_dict['observations'] = prior_gp
    
    return tfd.JointDistributionNamed(prior_dict)

In [6]:
def target_log_prob_from_joint(joint, Y_train):

    # Use `tf.function` to trace the loss for more efficient evaluation.
    @tf.function(autograph=False, experimental_compile=False)
    def target_log_prob(smooth_amplitude,
                       smooth_length_scale,
                       periodic_amplitude,
                       periodic_length_scale,
                       periodic_period,
                       periodic_local_amplitude,
                       periodic_local_length_scale,
                       global_periodic_amplitude,
                       global_periodic_length_scale,
                       global_periodic_period,
                       irregular_amplitude,
                       irregular_length_scale,
                       irregular_scale_mixture,
                       matern_onehalf_amplitude,
                       matern_onehalf_length_scale,
                       matern_threehalves_amplitude,
                       matern_threehalves_length_scale,
                       matern_fivehalves_amplitude,
                       matern_fivehalves_length_scale,
                       observation_noise_variance):
        
        return joint.log_prob({
          'smooth_amplitude': smooth_amplitude,
          'smooth_length_scale': smooth_length_scale,
          'periodic_amplitude': periodic_amplitude,
          'periodic_length_scale': periodic_length_scale,
          'periodic_period': periodic_period,
          'periodic_local_amplitude': periodic_local_amplitude,
          'periodic_local_length_scale': periodic_local_length_scale,
          'global_periodic_amplitude': global_periodic_amplitude,
          'global_periodic_length_scale': global_periodic_length_scale,
          'global_periodic_period': global_periodic_period,
          'irregular_amplitude': irregular_amplitude,
          'irregular_length_scale': irregular_length_scale,
          'irregular_scale_mixture': irregular_scale_mixture,
          'matern_onehalf_amplitude': matern_onehalf_amplitude,
          'matern_onehalf_length_scale': matern_onehalf_length_scale,
          'matern_threehalves_amplitude': matern_threehalves_amplitude,
          'matern_threehalves_length_scale': matern_threehalves_length_scale,
          'matern_fivehalves_amplitude': matern_fivehalves_amplitude,
          'matern_fivehalves_length_scale': matern_fivehalves_length_scale,
          'observation_noise_variance': observation_noise_variance,
          'observations': Y_train
        })
    
    return target_log_prob

In [7]:
with tf.device('/gpu:0'):
    # Speed up sampling by tracing with `tf.function`.
    @tf.function(autograph=False, experimental_compile=False)#, experimental_relax_shapes=True)
    def do_sampling(adaptive_sampler, initial_state, num_results=tf.constant(100), num_burnin_steps=tf.constant(500), parallel_iterations=tf.constant(10)):

        return tfp.mcmc.sample_chain(
          kernel=adaptive_sampler,
          current_state=initial_state,
          num_results=num_results,
          num_burnin_steps=num_burnin_steps,
          parallel_iterations=parallel_iterations,
          trace_fn=lambda current_state, kernel_results: kernel_results)

In [8]:
def theta_to_posterior(theta):
    
    if theta is None:
        return None
    #print("Actually constructing the posterior!")
    
    posterior = {
        'smooth_amplitude': tfd.Empirical(theta[0]),
        'smooth_length_scale': tfd.Empirical(theta[1]),
        'periodic_amplitude': tfd.Empirical(theta[2]),
        'periodic_length_scale': tfd.Empirical(theta[3]),
        'periodic_period': tfd.Empirical(theta[4]),
        'periodic_local_amplitude': tfd.Empirical(theta[5]),
        'periodic_local_length_scale': tfd.Empirical(theta[6]),
        'global_periodic_amplitude': tfd.Empirical(theta[7]),
        'global_periodic_length_scale': tfd.Empirical(theta[8]),
        'global_periodic_period': tfd.Empirical(theta[9]),
        'irregular_amplitude': tfd.Empirical(theta[10]),
        'irregular_length_scale': tfd.Empirical(theta[11]),
        'irregular_scale_mixture': tfd.Empirical(theta[12]),
        'matern_onehalf_amplitude': tfd.Empirical(theta[13]),
        'matern_onehalf_length_scale': tfd.Empirical(theta[14]),
        'matern_threehalves_amplitude': tfd.Empirical(theta[15]),
        'matern_threehalves_length_scale': tfd.Empirical(theta[16]),
        'matern_fivehalves_amplitude': tfd.Empirical(theta[17]),
        'matern_fivehalves_length_scale': tfd.Empirical(theta[18]),
        'observation_noise_variance': tfd.Empirical(theta[19]),
    }
    
    return posterior

In [9]:
def sample(X, Y, step_size, num_results, num_leapfrog_steps, num_burnin_steps, constrain_positive, target_accept_prob, parallel_iterations, theta=None):
    
    posterior = theta_to_posterior(theta)
    
    # Create Joint Model
    joint = prior_joint_model(X, Y, posterior)
    
    # Target Log Prob
    target_log_prob = target_log_prob_from_joint(joint, Y)

    # Create sampler
    sampler = tfp.mcmc.TransformedTransitionKernel(
        tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=target_log_prob,
            step_size=tf.cast(step_size, tf.float64),
            num_leapfrog_steps=num_leapfrog_steps),
            bijector=[constrain_positive]*20)

    adaptive_sampler = tfp.mcmc.DualAveragingStepSizeAdaptation(
        inner_kernel=sampler,
        num_adaptation_steps=int(0.8 * num_burnin_steps),
        target_accept_prob=tf.cast(target_accept_prob, tf.float64))

    # Initial States for pyhisical walk
    if theta == None:
        nitial_state = tf.constant([1.]*20, tf.float64)
        initial_state = [tf.cast(x, tf.float64) for x in [1.]*20]
    else:
        #initial_state = tf.constant([x[0] for x in theta], tf.float64)
        initial_state = [tf.cast(x[0], tf.float64) for x in theta]
        
    #initial_state = tf.constant(initial_state, tf.float64)
    
    samples, kernel_results = do_sampling(adaptive_sampler,
                                              initial_state,
                                              num_results=num_results,
                                              num_burnin_steps=num_burnin_steps,
                                              parallel_iterations=parallel_iterations)
    
    return samples, kernel_results

In [10]:
def simulate_slice(sslice, theta=None, padding=27_000//48//2, n=100, num_leapfrog_steps=8, step_size=0.1, num_results=100,
                   num_burnin_steps=200, parallel_iterations=10, target_accept_prob=0.75, predictive_noise_variance=0., num_predictive_points=200):
    
    
    # Preparation
    X=sslice["Recording"]["ContractionNoNorm"]
    Y=sslice["Recording"]["RrInterval"]
    X = np.array(X).astype(float).reshape(-1, 1)
    Y = np.array(Y)
    
    ### TEST
    X = tf.constant(X)
    Y = tf.constant(Y)
    ### TEST
    
    constrain_positive = tfb.Shift(np.finfo(np.float64).tiny)(tfb.Exp())
    
    #print("Starting Trial and Error")
    
    for i in range(5):
        try:
            samples, kernel_results = sample(X, Y,
                                     step_size,
                                     num_results,
                                     num_leapfrog_steps,
                                     num_burnin_steps,
                                     constrain_positive,
                                     target_accept_prob,
                                     parallel_iterations,
                                     theta=theta)
            break
        except tf.errors.InvalidArgumentError:
            if i >= 4:
                print(f"Cholesky decompositon error detected 5 times now. Trying one last time with inital priors.")
                samples, kernel_results = sample(X, Y,
                                                 step_size,
                                                 num_results,
                                                 num_leapfrog_steps,
                                                 num_burnin_steps,
                                                 constrain_positive,
                                                 target_accept_prob,
                                                 parallel_iterations,
                                                 theta=None)
                break
            print(f"Cholesky decompositon error detected in run {i+1}. Retrying.")                            
            #continue
            #break
    
    #print("End")
            

   # print("Cholesky Error detected 3 times, trying one last time withtaking default priors.")
   #             samples, kernel_results = sample(X, Y,
   #                                      step_size,
   #                                      num_results,
   #                                      num_leapfrog_steps,
   #                                      num_burnin_steps,
   #                                      constrain_positive,
   #                                      target_accept_prob,
   #                                      parallel_iterations,
   #                                      theta=None)

    (smooth_amplitude_samples,
    smooth_length_scale_samples,
    periodic_amplitude_samples,
    periodic_length_scale_samples,
    periodic_period_samples,
    periodic_local_amplitude_samples,
    periodic_local_length_scale_samples,
    global_periodic_amplitude_samples,
    global_periodic_length_scale_samples,
    global_periodic_period_samples,
    irregular_amplitude_samples,
    irregular_length_scale_samples,
    irregular_scale_mixture_samples,
    matern_onehalf_amplitude_samples,
    matern_onehalf_scale_samples,
    matern_threehalves_amplitude_samples,
    matern_threehalves_scale_samples,
    matern_fivehalves_amplitude_samples,
    matern_fivehalves_scale_samples,
    observation_noise_variance_samples) = samples
    
    # Given the posterior parameter and thus kernel samples, we can use that to fit the GP and predict for the posterior points
    predictive_index_points_ = np.linspace(0, padding-2, num_predictive_points, dtype=np.float64).reshape(-1, 1)
    
    with tf.device('/cpu:0'):

        # The sampled hyperparams have a leading batch dimension, `[num_results, ...]`,
        # so they construct a *batch* of kernels.
        batch_of_posterior_kernels = get_kernel_bayesian(smooth_amplitude_samples,
                                                    smooth_length_scale_samples,
                                                    periodic_amplitude_samples,
                                                    periodic_length_scale_samples,
                                                    periodic_period_samples,
                                                    periodic_local_amplitude_samples,
                                                    periodic_local_length_scale_samples,
                                                    global_periodic_amplitude_samples,
                                                    global_periodic_length_scale_samples,
                                                    global_periodic_period_samples,
                                                    irregular_amplitude_samples,
                                                    irregular_length_scale_samples,
                                                    irregular_scale_mixture_samples,
                                                    matern_onehalf_amplitude_samples,
                                                    matern_onehalf_scale_samples,
                                                    matern_threehalves_amplitude_samples,
                                                    matern_threehalves_scale_samples,
                                                    matern_fivehalves_amplitude_samples,
                                                    matern_fivehalves_scale_samples)

        # The batch of kernels creates a batch of GP predictive models, one for each
        # posterior sample.
        batch_gprm = tfd.GaussianProcessRegressionModel(
            mean_fn=lambda x: np.mean(Y),
            kernel=batch_of_posterior_kernels,
            index_points=predictive_index_points_,
            observation_index_points=X,
            observations=Y,
            observation_noise_variance=observation_noise_variance_samples,
            predictive_noise_variance=predictive_noise_variance)

        # To construct the marginal predictive distribution, we average with uniform
        # weight over the posterior samples.
        predictive_gprm = tfd.MixtureSameFamily(
            mixture_distribution=tfd.Categorical(logits=tf.zeros([num_results])),
            components_distribution=batch_gprm)

        simulated = predictive_gprm.sample(n)
    
    return simulated, samples

## Data 

In [11]:
def remove_outliers_in_recordings(recordings, in_seconds=True, low_rri: int = 300, high_rri: int = 2000, ectopic_beats_removal_method: str = "kamath"):
    
    cleared_recordings = []
    recordings_deepcopy = copy.deepcopy(recordings)
    denom = 1000 if in_seconds else 1
    
    for i, recording in enumerate(recordings_deepcopy):
        
        # Status
        #print(str(round(i/len(recordings_deepcopy)*100, 2)) + "%", end="\r")
        
        rrinterval = hrv.remove_outliers(recording["Recording"]["RrInterval"],
                                                               low_rri=low_rri, high_rri=high_rri, verbose=False)
        
        rrinterval = hrv.remove_ectopic_beats(rrinterval,
                                        method=ectopic_beats_removal_method, verbose=False)
        
        recording["Recording"]["RrInterval"] = np.array(rrinterval) / denom
        
        recording["Recording"].dropna(inplace=True)
        
        cleared_recordings.append(recording)
        
    return cleared_recordings

In [12]:
def c_splice_lod_constant_by_number(lod, n=48):
    spliced_recordings = []
    
    for i, recording in enumerate(lod):
        
        # Status
        #print(str(round(i/len(lod)*100, 2)) + "%", end="\r")
        
        splices = np.array_split(recording["Recording"], n)
        
        for splice in splices:
            recording_deepcopy = copy.deepcopy(recording)
            recording_deepcopy["Recording"] = splice
            recording_deepcopy["Recording"]["ContractionNoNorm"] = list(range(len(recording_deepcopy["Recording"]["ContractionNoNorm"])))
            spliced_recordings.append(recording_deepcopy)
    
    return spliced_recordings

In [13]:
# Imports
import numpy as np

import utilities as utils
from utilities import *

In [14]:
# Parameters
input_gdansk = "../data/age_decades/"
input_physionet = "/home/flennic/Downloads/physionet.org/files/crisdb/1.0.0/"
subdirs_physionet = ["e/", "f/", "m/"]
output_dir = "../data/preprocessed/"

data_source_type = "gdansk" # ["gdansk", "physionet"]
splice_type = "constant" # ["complete", "constant", "random"]
label_type = "regression" # ["classification", "regression"]
output_type = "deep" # ["features", "deep"]
in_seconds = True # Will be automatically set to false for creating features, as the frequency features require milli seconds.

splits_gdansk = [0.6, 0.2, 0.2]
splits_physionet = [0.8, 0.1, 0.1]
seed = 42
pad_length = 27_000 if data_source_type == "gdansk" else 135_000 # For deep learning padding
N = 48 if data_source_type == "gdansk" else 240

In [15]:
# Auto adjustment
if data_source_type == "gdansk":
    splits = splits_gdansk
elif data_source_type == "physionet":
    splits = splits_physionet
else:
    raise Exception("Data source not supported.")
    
if output_type == "features":
    in_seconds = False
    unit = "milliseconds"
else:
    unit = "seconds" if in_seconds else "milliseconds"
    
if splice_type == "constant":
    pad_length //= N
    
np.random.seed(seed=seed)

In [16]:
%%time
if data_source_type == "gdansk":
    recordings = utils.read_gdansk_to_dict(input_gdansk)    
elif data_source_type == "physionet":
    recordings = utils.read_physionet_to_dict(input_physionet, subdirs_physionet)
else:
    raise Exception("Data source not supported.")

CPU times: user 13.3 s, sys: 71.3 ms, total: 13.4 s
Wall time: 13.3 s


%%time
cleared_recordings = remove_outliers_in_recordings(recordings, in_seconds=in_seconds)

In [17]:
%%time
train_recordings, val_recordings, test_recordings = np.array_split(recordings, (np.array(splits)[:-1].cumsum() * len(recordings)).astype(int))

CPU times: user 0 ns, sys: 138 µs, total: 138 µs
Wall time: 134 µs


In [18]:
train_recordings_cleared = remove_outliers_in_recordings(train_recordings)
val_recordings_cleared = remove_outliers_in_recordings(val_recordings)
test_recordings_cleared = remove_outliers_in_recordings(test_recordings)

In [19]:
%%time
train_recordings = c_splice_lod_constant_by_number(train_recordings, n=2*N)
val_recordings = c_splice_lod_constant_by_number(val_recordings, n=2*N)
test_recordings = c_splice_lod_constant_by_number(test_recordings, n=2*N)

del train_recordings_cleared, val_recordings_cleared, test_recordings_cleared

CPU times: user 11.4 s, sys: 87.6 ms, total: 11.4 s
Wall time: 11.4 s


len(train_recordings)

train_recordings[0]

X = train_recordings[0]["Recording"]["ContractionNoNorm"]
Y = train_recordings[0]["Recording"]["RrInterval"]
X2 = train_recordings[1]["Recording"]["ContractionNoNorm"]
Y2 = train_recordings[1]["Recording"]["RrInterval"]
given_slice = pd.DataFrame({"X": X, "Y": Y})
given_slice2 = pd.DataFrame({"X": X2, "Y": Y2})

import seaborn as sns
sns.lineplot(X, Y);
sns.lineplot(X2, Y2);

%%time
simulated_data, theta = simulate_slice(given_slice)

sns.lineplot(X, Y);
sns.lineplot(range(len(simulated_data[0,:])), simulated_data[5,:]);

%%time
simulated_data_Post, theta_Post = simulate_slice(given_slice, theta)

sns.lineplot(X, Y);
sns.lineplot(range(len(simulated_data[0,:])), simulated_data_Post[5,:]);

%%time
simulated_data_Post2, theta_Post2 = simulate_slice(given_slice, theta_Post)

sns.lineplot(X, Y);
sns.lineplot(range(len(simulated_data[0,:])), simulated_data_Post2[5,:]);

%%time
simulated_data2, theta2 = simulate_slice(given_slice2, theta)

simulated_data2_None, theta2_None = simulate_slice(given_slice2, None)

WARNING:tensorflow:5 out of the last 5 calls to <function do_sampling at 0x7fa8f33ed170> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings is likely due to passing python objects instead of tensors. Also, tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. Please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for more details.

sns.lineplot(range(len(simulated_data2[0,:])), simulated_data2[0,:]);
sns.lineplot(range(len(simulated_data2_None[0,:])), simulated_data2_None[0,:]);

In [20]:
def simulation_generator(recordings, padding=27_000//48//2,
                                     n=100,
                                     num_leapfrog_steps=8,
                                     step_size=0.1,
                                     num_results=100,
                                     num_burnin_steps=200,
                                     parallel_iterations=10,
                                     target_accept_prob=0.75,
                                     predictive_noise_variance=0.,
                                     num_predictive_points=200):
    
    theta = None
    
    for recording in recordings:
        simulated_data, theta = simulate_slice(recording,
                                               theta=theta,
                                               padding=padding,
                                               n=n,
                                               num_leapfrog_steps=num_leapfrog_steps,
                                               step_size=step_size,
                                               num_results=num_results,
                                               num_burnin_steps=num_burnin_steps,
                                               parallel_iterations=parallel_iterations,
                                               target_accept_prob=target_accept_prob,
                                               predictive_noise_variance=predictive_noise_variance,
                                               num_predictive_points=num_predictive_points)
        yield recording, simulated_data

In [21]:
import subprocess

def file_len(fname):
    p = subprocess.Popen(['wc', '-l', fname], stdout=subprocess.PIPE, 
                                              stderr=subprocess.PIPE)
    result, err = p.communicate()
    if p.returncode != 0:
        raise IOError(err)
    return int(result.strip().split()[0])

In [22]:
def append_df_list_to_files(dfs, basedir, set_selector="train", padding=200, append=False, start_index=0):

    append_df_list_to_file_feature(dfs, basedir, set_selector, append, start_index)
    append_df_list_to_file_deep(dfs, basedir, set_selector, padding, append, start_index)

In [23]:
def append_df_list_to_file_feature(dfs, basedir, set_selector="train", append=False, start_index=0):
    
    header = not append
    mode = 'a' if append else 'w'
    indeces = list(range(start_index, start_index+len(dfs)))
    
    for df in dfs:
        df["Recording"]["RrInterval"] = df["Recording"]["RrInterval"] * 1000.0
        
    #print(dfs[0])
    
    dfs_classification = utils.recordings_to_feature_dataframe(dfs, classification=True)
    dfs_regression = utils.recordings_to_feature_dataframe(dfs, classification=False)
    
    dfs_classification.index = indeces
    dfs_regression.index = indeces
    
    dfs_classification.to_csv(f'{basedir}gdansk_simulated_constant_features_classification_milliseconds_original_{set_selector}.csv', mode=mode, header=header)
    dfs_regression.to_csv(f'{basedir}gdansk_simulated_constant_features_regression_milliseconds_original_{set_selector}.csv', mode=mode, header=header)

In [24]:
def append_df_list_to_file_deep(dfs, basedir, set_selector="train", padding=200, append=False, start_index=0):
    
    header = not append
    mode = 'a' if append else 'w'
    indeces = list(range(start_index, start_index+len(dfs)))
    #print(indeces)
    
    dfs_classification = utils.recordings_to_deep_dataframe(dfs, pad_length=padding, classification=True, gdansk=True)
    dfs_regression = utils.recordings_to_deep_dataframe(dfs, pad_length=padding, classification=False, gdansk=True)
    
    dfs_classification.index = indeces
    dfs_regression.index = indeces
    
    dfs_classification.to_csv(f'{basedir}gdansk_simulated_constant_deep_classification_seconds_original_{set_selector}.csv', mode=mode, header=header)
    dfs_regression.to_csv(f'{basedir}gdansk_simulated_constant_deep_regression_seconds_original_{set_selector}.csv', mode=mode, header=header)

In [25]:
def tensor_1d_to_recording(tensor, start_index=0):
    
    series = pd.Series(tensor)
    indeces = list(range(start_index, start_index+len(series)))
    
    df = pd.DataFrame({"ContractionNo": list(range(len(series))),
                       "ContractionNoNorm": list(range(len(series))),
                       "RrInterval": series})
    
    df.index = indeces

    return df

tensor_1d_to_recording(tf.constant([[0,1,2,3,4,5,6,7,8,9], [10,11,12,13,14,15,16,17,18,19]])[1,:], start_index=100)

In [26]:
# This is just a memory leak test.
def simulate_data_memory_leak(recordings):
    my_generator = simulation_generator(recordings, n=100)
    for i, item in enumerate(my_generator):
        print(i)

In [27]:
import os.path
from os import path
import time
from datetime import datetime, timedelta

#sslice, theta=None, padding=27_000//48//2, n=100, num_leapfrog_steps=8, step_size=0.1, num_results=100,
#                   num_burnin_steps=200, parallel_iterations=10, target_accept_prob=0.75, predictive_noise_variance=0., num_predictive_points=200
from memory_profiler import profile
@profile(precision=6)
def simulate_data(orig_recordings,
                  set_selector="train",
                  basedir="../data/preprocessed/",
                  padding=27_000//48//2,
                  num_leapfrog_steps=8,
                  step_size=0.1,
                  n=10,
                  num_results=100,
                  num_burnin_steps=100,
                  parallel_iterations=10,
                  target_accept_prob=0.75,
                  predictive_noise_variance=0.,
                  num_predictive_points=200):
    
    path_feature_classification = f"{basedir}gdansk_simulated_constant_features_classification_milliseconds_original_{set_selector}.csv"
    path_feature_regression = f"{basedir}gdansk_simulated_constant_features_regression_milliseconds_original_{set_selector}.csv"
    path_deep_classification = f"{basedir}gdansk_simulated_constant_deep_classification_seconds_original_{set_selector}.csv"
    path_deep_regression = f"{basedir}gdansk_simulated_constant_deep_regression_seconds_original_{set_selector}.csv"
    
    output_paths = [path_feature_classification, path_feature_regression, path_deep_classification, path_deep_regression]
    #print(output_paths)
    
    # Determine where to start
    start = 0
    if all(map(path.exists, output_paths)):
        # Continue
        #print("CONTINUEE")
        file_lengths = list(map(file_len, output_paths))
        #print(file_lengths)
        if len(set(file_lengths)) == 1:
            start = list(file_lengths)[0]//n
    
    append = start != 0
    total_count = len(orig_recordings)
    
    print(f"Processing {start+1}/{total_count}")
    
    # If not everything is sane, raise an exception
    if os.path.exists(path_feature_classification) and start == 0:
        raise Exception(f"For safety reasons, file {path_feature_classification} must be deleted manually.")
        
    if os.path.exists(path_feature_regression) and start == 0:
        raise Exception(f"For safety reasons, file {path_feature_regression} must be deleted manually.")
        
    if os.path.exists(path_deep_classification) and start == 0:
        raise Exception(f"For safety reasons, file {path_deep_classification} must be deleted manually.")
        
    if os.path.exists(path_deep_regression) and start == 0:
        raise Exception(f"For safety reasons, file {path_deep_regression} must be deleted manually.")
    
    orig_recordings = orig_recordings[start:]
    
    start_time = time.time()
    
    for i, res in enumerate(simulation_generator(orig_recordings, n=n)):

        orig_recording, simulated_data = res
        
        #print(orig_recording)
        #print(simulated_data)
        print("Data simulated, creating list of dataframes.")
        
        simulated_dfs = []
        
        for j in range(simulated_data.shape[0]):
            
            simulated = simulated_data[j,:]
            
            recording_deepcopy = copy.deepcopy(orig_recording)
            recording_deepcopy["Recording"] = tensor_1d_to_recording(simulated)
            simulated_dfs.append(recording_deepcopy)
        
        print("List of dataframes created. Saving results to file.")
        # 2 + 9*10
        #print(f"{start*n} + {i}*{n}")
        index_continue = start*n + i*n
        #print(f"##### {index_continue}")
        print(simulated_dfs[0])
        append_df_list_to_files(simulated_dfs, basedir, set_selector, padding=200, append=append, start_index=index_continue)
        
        time_diff = time.time() - start_time
        time_remaining_in_hours = (time_diff*(len(orig_recordings)-i))//3600
        
        print(f"Time taken: {round(time_diff)} seconds")
        print(f"Estimated time remaining: {round(time_remaining_in_hours)} hours")
        print(f"Estimated time finished: {datetime.now() + timedelta(hours=time_remaining_in_hours)}")
        print("\n\n")
        print(f"Processing {start+(i+2)}/{total_count} next, if available, or end.")
        
        append=True
        
        start_time = time.time()
        
        if i >= 20:
            return

train_recordings[0]

In [28]:
set_selector = "train" # ["train", "val", "test"]
n = 100

In [29]:
len(train_recordings[64]['Recording']['RrInterval']) # 227

227

In [30]:
train_recordings[63]['Recording']['RrInterval']

14311    1208.0
14312    1232.0
14313    1224.0
14314    1224.0
14315    1240.0
          ...  
14533    1256.0
14534    1264.0
14535    1256.0
14536    1248.0
14537    1264.0
Name: RrInterval, Length: 227, dtype: float64

In [31]:
train_recordings[64]['Recording']['RrInterval']

14538    1272.0
14539    1296.0
14540    1280.0
14541    1280.0
14542    1296.0
          ...  
14760    1192.0
14761    1192.0
14762    1200.0
14763    1200.0
14764    1208.0
Name: RrInterval, Length: 227, dtype: float64

In [ ]:
%%time
simulate_data(train_recordings, set_selector=set_selector, n=n)

ERROR: Could not find file <ipython-input-27-2e0703cb0726>
NOTE: %mprun can only be used on functions defined in physical files, and not in the IPython environment.
Processing 437/10368


In [ ]:
#simulate_data_memory_leak(train_recordings)